In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Data Extraction

In [ ]:
!pip install requests
!pip install html5lib
!pip install bs4
!pip install textstat
!pip install XlsxWriter

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import os
import nltk
from nltk.tokenize import word_tokenize
import string
import spacy
import textstat
from textstat.textstat import textstatistics
import xlsxwriter

nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
file_path = '/content/drive/My Drive/Test_Assignment/Input.xlsx'
df = pd.read_excel(file_path)
print(df.sample(5))

             URL_ID                                                URL
38  blackassign0039  https://insights.blackcoffer.com/how-to-protec...
74  blackassign0075  https://insights.blackcoffer.com/how-to-overco...
60  blackassign0061  https://insights.blackcoffer.com/management-ch...
45  blackassign0046  https://insights.blackcoffer.com/all-you-need-...
64  blackassign0065  https://insights.blackcoffer.com/will-we-ever-...


In [ ]:
save_directory = '/content/drive/My Drive/Test_Assignment/WebScraped_Data/'
global fetch_url_404
fetch_url_404 = []
for index, row in df.iterrows():
    url = row['URL']
    urlid = row['URL_ID']

    try:
        response = requests.get(url)
        if response.status_code == 200:

            soup = BeautifulSoup(response.content, 'html.parser')
            txt = soup.get_text()
            # Remove extra spaces and normalize whitespace
            cleaned_text = re.sub(r'\n\s*\n', '.\n', txt).strip()
            # Saving every website data in a text file with URL_ID as its file name
            filename = f"{urlid}.txt"
            filepath = os.path.join(save_directory, filename)
            with open(filepath, 'w', encoding='utf-8') as file:

                file.write(cleaned_text)
                print(f"Web content for {urlid} saved successfully at {filepath}.")
            #print(f"Title of {url}: {soup.title.get_text()}")

        else:
            print(f"Failed to fetch {url}. Status code: {response.status_code}")
            fetch_url_404.append(urlid)
    except Exception as e:
        print(f"An error occurred while processing {url}: {e}")


In [ ]:
stop_words_files = ['/content/drive/MyDrive/Test_Assignment/StopWords/StopWords_Auditor.txt',
                    '/content/drive/MyDrive/Test_Assignment/StopWords/StopWords_Currencies.txt',
                    '/content/drive/MyDrive/Test_Assignment/StopWords/StopWords_DatesandNumbers.txt',
                    '/content/drive/MyDrive/Test_Assignment/StopWords/StopWords_Generic.txt',
                    '/content/drive/MyDrive/Test_Assignment/StopWords/StopWords_GenericLong.txt',
                    '/content/drive/MyDrive/Test_Assignment/StopWords/StopWords_Geographic.txt',
                    '/content/drive/MyDrive/Test_Assignment/StopWords/StopWords_Names.txt']

In [ ]:
# Read Stop Words Files

stop_words = set()

for file_name in stop_words_files:
    with open(file_name, 'r', encoding='latin-1') as file:
        words = file.read().split()
        stop_words.update(words)

In [ ]:
def RemovePunctuationandTokenize(text):
  text_without_punctuation = text.translate(str.maketrans('', '', string.punctuation))
  tokens = word_tokenize(text_without_punctuation)
  return tokens

In [ ]:
# Function to count complex words in text
def count_complex_words(text):
  words = text.split()
  complex_word_count = 0

  for word in words:
      syllable_count = textstat.syllable_count(word)
      if syllable_count >= 3:
          complex_word_count += 1

  return complex_word_count



In [ ]:
# Function to count personal pronouns
def count_personal_pronouns(text):
    # Define the pattern for personal pronouns excluding "US" as a country
    pattern = r'\b(?:I|we|my|ours|us)\b(?![a-z])'

    # Find all matches of personal pronouns in the text
    matches = re.findall(pattern, text, flags=re.IGNORECASE)

    # Return the count of personal pronouns
    return len(matches)


In [ ]:
# Syllable Count Per Word
# Function to count syllables in a word
def count_syllables(word):
    vowels = "aeiouy"
    count = 0
    endings = ["es", "ed"]  # Exceptions for syllable counting

    word = word.lower().strip(".:;?!")

    if len(word) > 0:  # Check if word is not empty
        if word[-2:] in endings:  # Handle exceptions for syllable counting
            pass
        else:
            if word[0] in vowels:
                count += 1
            for index in range(1, len(word)):
                if word[index] in vowels and word[index - 1] not in vowels:
                    count += 1
            if word.endswith("e"):
                count -= 1  # Exclude silent 'e' at the end

    return max(1, count)  # At least one syllable per word


In [ ]:
# Function to calculate average word length
def calculate_average_word_length(text):
    # Split the text into words
    words = text.split()

    # Calculate total number of characters in all words
    total_characters = sum(len(word) for word in words)

    # Calculate total number of words
    total_words = len(words)

    # Calculate average word length
    average_length = total_characters / total_words if total_words > 0 else 0

    return average_length


In [ ]:

# Fetch and preprocess each text file in the directory
save_directory = '/content/drive/My Drive/Test_Assignment/WebScraped_Data/'

global excel_data
excel_data = list()
for filename in os.listdir(save_directory):
    if filename.endswith(".txt"):
        file_path = os.path.join(save_directory, filename)
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()

                # Remove Punctuation and tokenize data
                preprocessed_content = RemovePunctuationandTokenize(content)

                # Remove Stop Words
                filtered_tokens = [token for token in preprocessed_content if token.lower() not in stop_words]

                # Positive Score
                positive_list_file = ['/content/drive/MyDrive/Test_Assignment/MasterDictionary/positive-words.txt']
                pos_words = set()
                Positive_Score = 0
                for file_name in positive_list_file:
                    with open(file_name, 'r', encoding='latin-1') as file:
                        words = file.read().split()
                        pos_words.update(words)

                for pos in pos_words:

                  if pos in filtered_tokens:
                    Positive_Score = Positive_Score + 1

                print("Positive Score : " , Positive_Score)

                # Negative Score
                negative_list_file = ['/content/drive/MyDrive/Test_Assignment/MasterDictionary/negative-words.txt']

                neg_words = set()
                Negative_Score = 0
                for file_name in negative_list_file:
                    with open(file_name, 'r', encoding='latin-1') as file:
                        words = file.read().split()
                        neg_words.update(words)

                for neg in neg_words:

                  if neg in filtered_tokens:
                    Negative_Score = Negative_Score + 1

                print("Negative Score : ", Negative_Score)

                # Polarity Score
                # Polarity Score = (Positive Score – Negative Score)/ ((Positive Score + Negative Score) + 0.000001)

                Polarity_Score = (Positive_Score - Negative_Score)/ ((Positive_Score + Negative_Score) + 0.000001)
                print("Polarity Score : ", Polarity_Score)

                # Subjectivity Score
                # Subjectivity Score = (Positive Score + Negative Score)/ ((Total Words after cleaning) + 0.000001)
                num_token = len(filtered_tokens)
                Subjectivity_Score = (Positive_Score + Negative_Score)/ ((num_token) + 0.000001)
                print("Subjectivity Score : ", Subjectivity_Score)

                ########### Analysis of Readability ###########
                # Avg Sentence length

                # number of words
                num_word = len(preprocessed_content)

                # number of sentences
                sentences = content.split('.')
                num_sentences = len(sentences)

                # Adjust for cases where empty strings might cause incorrect counting
                if content.endswith('.'):
                    num_sentences -= 1
                # Average Sentence Length = the number of words / the number of sentences
                Average_Sentence_Length = num_word / num_sentences
                print("Average Sentence Length : ", Average_Sentence_Length)


                # Count complex words
                complex_words_count = count_complex_words(content)

                # Display the number of complex words
                print("Number of complex words:", complex_words_count)

                # Percentage of Complex words = the number of complex words / the number of words
                # number of complex words

                Percentage_of_complex_words = complex_words_count / num_word
                print("Percentage of Complex words : ", Percentage_of_complex_words)

                # Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)
                Fog_Index = 0.4 * (Average_Sentence_Length + Percentage_of_complex_words)
                print("Fog Index : ", Fog_Index)

                # Average Number of Words Per Sentence = the total number of words / the total number of sentences
                Average_Number_of_Words_Per_Sentence = num_word / num_sentences
                print("Average Number of Words Per Sentence : ", Average_Number_of_Words_Per_Sentence)

                # Split the cleaned text into words
                words = content.split()

                # Calculate syllable count per word
                syllable_count_per_word = {word: count_syllables(word) for word in words if word.strip()}

                # ------------------------------------------------------------------------------#
                # Display syllable count per word
                sum_of_syllable = 0
                for word, syllable_count in syllable_count_per_word.items():
                    sum_of_syllable = sum_of_syllable + syllable_count
                    #print(f"Word: {word} | Syllable Count: {syllable_count}")
                avg_syllable = sum_of_syllable / num_word
                # ------------------------------------------------------------------------------#

                # Calculate count of personal pronouns in the sample text
                pronoun_count = count_personal_pronouns(content)

                # Display the count of personal pronouns
                print("Count of personal pronouns:", pronoun_count)

                ###########  Average Word Length  ###########

                # Calculate average word length in the sample text
                avg_word_length = calculate_average_word_length(content)

                # Display the average word length
                print("Average Word Length:", avg_word_length)

                # Creating list of directory to save in excel file

                data = {
                              # 'URL_ID' : 'filename',
                              # 'URL' : url,
                              'POSITIVE SCORE' : Positive_Score,
                              'NEGATIVE SCORE' : Negative_Score,
                              'POLARITY SCORE' : Polarity_Score,
                              'SUBJECTIVITY SCORE' : Subjectivity_Score,
                              'AVERAGE SENTENCE LENGTH' : Average_Sentence_Length,
                              'PERCENTAGE OF COMPLEX WORDS' : Percentage_of_complex_words,
                              'FOG INDEX' : Fog_Index,
                              'AVERAGE NUMBER OF WORDS PER SENTENCE' : Average_Number_of_Words_Per_Sentence,
                              'COMPLEX WORD COUNT' : complex_words_count,
                              'WORD COUNT' : num_word,
                              'SYLLABLE PER WORD' : avg_syllable,
                              'PERSONAL PRONOUNS' : pronoun_count,
                              'AVG WORD LENGTH' : avg_word_length
                          }
                excel_data.append(data)
                print(excel_data)


        except Exception as e:
            print(f"An error occurred while processing {filename}: {e}")


In [ ]:
import openpyxl

# Load the existing workbook using xlsxwriter
existing_file = '/content/drive/MyDrive/Test_Assignment/Output Data Structure.xlsx'

workbook = openpyxl.load_workbook(existing_file)
worksheet = workbook.active  # Access the active worksheet

In [ ]:
print(fetch_url_404)
print(len(excel_data))
print(len(worksheet['A'])-14)

['blackassign0036', 'blackassign0049']
98
101


In [ ]:
# Get values from column A and check for each target ID
for index , cell in enumerate(worksheet['A'] , start = 1):

  if cell.value in fetch_url_404:

        worksheet.cell(index , 3 , 'NA')
        worksheet.cell(index , 4 , 'NA')
        worksheet.cell(index , 5 , 'NA')
        worksheet.cell(index , 6 , 'NA')
        worksheet.cell(index , 7 , 'NA')
        worksheet.cell(index , 8 , 'NA')
        worksheet.cell(index , 9 , 'NA')
        worksheet.cell(index , 10 , 'NA')
        worksheet.cell(index , 11 , 'NA')
        worksheet.cell(index , 12 , 'NA')
        worksheet.cell(index , 13 , 'NA')
        worksheet.cell(index , 14 , 'NA')
        worksheet.cell(index , 15 , 'NA')
  else:
        if index <= len(excel_data):
              entry = excel_data[index - 1]  # Get the corresponding entry
              worksheet.cell(index+1, 3, entry.get('POSITIVE SCORE'))
              worksheet.cell(index+1, 4, entry.get('NEGATIVE SCORE'))
              worksheet.cell(index+1 , 5 , entry.get('POLARITY SCORE'))
              worksheet.cell(index+1 , 6 , entry.get('SUBJECTIVITY SCORE'))
              worksheet.cell(index+1 , 7 , entry.get('AVERAGE SENTENCE LENGTH'))
              worksheet.cell(index+1 , 8 , entry.get('PERCENTAGE OF COMPLEX WORDS'))
              worksheet.cell(index+1 , 9 , entry.get('FOG INDEX'))
              worksheet.cell(index+1 , 10 , entry.get('AVERAGE NUMBER OF WORDS PER SENTENCE'))
              worksheet.cell(index+1 , 11 , entry.get('COMPLEX WORD COUNT'))
              worksheet.cell(index+1 , 12 , entry.get('WORD COUNT'))
              worksheet.cell(index+1 , 13 , entry.get('SYLLABLE PER WORD'))
              worksheet.cell(index+1 , 14 , entry.get('PERSONAL PRONOUNS'))
              worksheet.cell(index+1 , 15 , entry.get('AVG WORD LENGTH'))
        else:
            # If index is out of excel_data range, write 'NA'
            for col_index in range(3, 16):
                worksheet.cell(index, col_index, 'NA')

workbook.save(existing_file)
